# Data wrangling
## 1. Create metatable
## 2. Divide TPM table into accession-sets

In [1]:
library(tidyverse)
set.seed(42) # For reproducibility of results
# import
setwd("/home/vanda/Documents/PanBarley_transcriptome/")
tpm <- read.table("PanBaRT20_tpm_genes.csv", sep = ",", row.names = 1, header = TRUE, stringsAsFactors = TRUE)
hogs <- read.table("PanBaRTv2_and_Ortholog_match_singlecopy.csv", sep = ",", header = TRUE, stringsAsFactors = TRUE)
head(tpm, 2)
head(hogs, 2)

── Attaching core tidyverse packages ─────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ───────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


,Akashinriki_Ca1,Akashinriki_Ca2,Akashinriki_Ca3,Akashinriki_Co1,Akashinriki_Co2,Akashinriki_Co3,Akashinriki_In1,Akashinriki_In2,Akashinriki_In3,Akashinriki_Ro1,⋯,RGTPlanet_Co3,RGTPlanet_In1,RGTPlanet_In2,RGTPlanet_In3,RGTPlanet_Ro1,RGTPlanet_Ro2,RGTPlanet_Ro3,RGTPlanet_Sh1,RGTPlanet_Sh2,RGTPlanet_Sh3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1H00001,4.007573,3.927487,3.833363,19.69449,26.30955,25.0926,4.080795,3.197684,6.416916,0.550499,⋯,18.30323,4.690004,4.166625,5.417613,1.272258,1.170904,1.360069,30.31152,27.36412,26.2589
chr1H00002,130.423413,95.909839,127.868675,606.02149,665.34842,636.3792,116.085727,77.322687,175.883190,16.129595,⋯,455.50814,101.854365,101.949697,130.543577,34.625894,32.760388,36.347136,888.63900,716.14744,670.2170


,PanBaRT20_gene_id,PanBaRT20_transcript_id,PSVCP20_pos_strand,GsRTD_gene_id,GsRTD_transcript_id,Source_data,Genotype,Pan_category,PanBaRT20_DE_tissue,Ortholog,Orth_category,Pan_Orth_category_match
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,chr1H00001,chr1H00001.1,chr1H:49468-50716:-,HOR13942_chr1HG04824,HOR13942_chr1HG04824.1,RNA,HOR13942,shell;single,Co;In,HOG0012447,core;single,No
2,chr1H00001,chr1H00001.1,chr1H:49468-50716:-,Barke_chr1HG02204,Barke_chr1HG02204.1,RNA,Barke,shell;single,Co;In,HOG0012447,core;single,No


In [2]:
dim(tpm)
dim(hogs)

[1] 79580   297

[1] 1464198      12

In [3]:
# subset truly 20let HOGs
hog <- hogs %>% 
        select(PanBaRT20_gene_id, GsRTD_gene_id, Ortholog) %>%
        distinct() # remove rows with duplicate values (filter transcript-based geneduplicates)
dim(hog)
# total number of HOGs is 408,809
hog20 <- hog %>% group_by(Ortholog, PanBaRT20_gene_id) %>% count() %>% filter(n == 20) %>% ungroup() %>% select(Ortholog) 
dim(hog20)
head(hog20)
# total number of 20let-HOGs is 11,829
hog20genes <- inner_join(hog20, hog, by ="Ortholog")
dim(hog20genes)
head(hog20genes)
write.csv(hog20genes, file = "HOG20singlecopy_conversion_geneID.csv", row.names = FALSE)
# create col of 20let geneIDs
hog20genesonly <- hog20genes %>% select(PanBaRT20_gene_id) %>% distinct()
dim(hog20genesonly)

[1] 408809      3

[1] 11829     1

Ortholog
<fct>
HOG0001021
HOG0001033
HOG0001269
HOG0001378
HOG0001383
HOG0001586


[1] 236589      3

Ortholog,PanBaRT20_gene_id,GsRTD_gene_id
<fct>,<fct>,<fct>
HOG0001021,chr6H66186,OUN333_chr6HG30399
HOG0001021,chr6H66186,ZDM01467_chr6HG29123
HOG0001021,chr6H66186,HOR10350_chr6HG30248
HOG0001021,chr6H66186,HOR13821_chr6HG29768
HOG0001021,chr6H66186,HOR21599_chr6HG31068
HOG0001021,chr6H66186,HOR3081_chr6HG30986


[1] 11838     1

In [4]:
# find out what causes difference
hog20 <- hog %>% group_by(Ortholog, PanBaRT20_gene_id) %>% count() %>% filter(n == 20)  %>% ungroup() %>% select(Ortholog)
dim(hog20)
# translate it to GsRTDs as well
hog20genes <- inner_join(hog20, hog, by ="Ortholog")
dim(hog20genes)
hog20gs_extras <- hog20genes %>% select(GsRTD_gene_id)
hog20extra <- hog20genes %>% select(GsRTD_gene_id) %>% distinct()
dim(hog20extra)
# find duplicated GsRTD_gene_ids
hog20gs_extras %>%
  group_by_all() %>%
  filter(n()>1) %>%
  ungroup()

[1] 11829     1

[1] 236589      3

[1] 236580      1

GsRTD_gene_id
<fct>
Igri_chr6HG28688
Igri_chr6HG28688
OUN333_chr2HG06674
OUN333_chr2HG06674
Morex_chr3HG16080
Morex_chr3HG16080
HOR9043_chr4HG20083
HOR9043_chr4HG20083
HOR7552_chr5HG22561


In [5]:
# find out duplicated rows
#dup1 <- hogs %>% filter(GsRTD_gene_id == "Igri_chr6HG28688") %>% distinct(PanBaRT20_gene_id)
dup1 <- hogs %>% filter(GsRTD_gene_id == "Igri_chr6HG28688") %>% distinct()
dup1
dup2 <- hogs %>% filter(GsRTD_gene_id == "OUN333_chr2HG06674") %>% distinct()
dup2

PanBaRT20_gene_id,PanBaRT20_transcript_id,PSVCP20_pos_strand,GsRTD_gene_id,GsRTD_transcript_id,Source_data,Genotype,Pan_category,PanBaRT20_DE_tissue,Ortholog,Orth_category,Pan_Orth_category_match
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
chr6H59649,chr6H59649.1,chr6H:123962966-123963433:-,Igri_chr6HG28688,Igri_chr6HG28688.1,ISO,Igri,cloud;single,NotDE,HOG0011400,core;single,No
chr6H59648,chr6H59648.2,chr6H:123962121-123970610:-,Igri_chr6HG28688,Igri_chr6HG28688.19,ISO&RNA,Igri,core;multiple,Co,HOG0011400,core;single,No
chr6H59648,chr6H59648.2,chr6H:123962121-123970610:-,Igri_chr6HG28688,Igri_chr6HG28688.20,ISO&RNA,Igri,core;multiple,Co,HOG0011400,core;single,No
chr6H59648,chr6H59648.2,chr6H:123962121-123970610:-,Igri_chr6HG28688,Igri_chr6HG28688.25,ISO&RNA,Igri,core;multiple,Co,HOG0011400,core;single,No
chr6H59648,chr6H59648.2,chr6H:123962121-123970610:-,Igri_chr6HG28688,Igri_chr6HG28688.37,ISO&RNA,Igri,core;multiple,Co,HOG0011400,core;single,No
chr6H59648,chr6H59648.2,chr6H:123962121-123970610:-,Igri_chr6HG28688,Igri_chr6HG28688.38,ISO&RNA,Igri,core;multiple,Co,HOG0011400,core;single,No
chr6H59648,chr6H59648.3,chr6H:123962121-123970610:-,Igri_chr6HG28688,Igri_chr6HG28688.43,ISO,Igri,core;multiple,Co,HOG0011400,core;single,No
chr6H59648,chr6H59648.3,chr6H:123962121-123970610:-,Igri_chr6HG28688,Igri_chr6HG28688.3,ISO,Igri,core;multiple,Co,HOG0011400,core;single,No
chr6H59648,chr6H59648.3,chr6H:123962121-123970610:-,Igri_chr6HG28688,Igri_chr6HG28688.4,ISO,Igri,core;multiple,Co,HOG0011400,core;single,No


PanBaRT20_gene_id,PanBaRT20_transcript_id,PSVCP20_pos_strand,GsRTD_gene_id,GsRTD_transcript_id,Source_data,Genotype,Pan_category,PanBaRT20_DE_tissue,Ortholog,Orth_category,Pan_Orth_category_match
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
chr2H15971,chr2H15971.1,chr2H:402864281-402873246:+,OUN333_chr2HG06674,OUN333_chr2HG06674.1,ISO&RNA,OUN333,core;multiple,In;Ro;Sh,HOG0014921,core;single,No
chr2H15971,chr2H15971.1,chr2H:402864281-402873246:+,OUN333_chr2HG06674,OUN333_chr2HG06674.4,ISO&RNA,OUN333,core;multiple,In;Ro;Sh,HOG0014921,core;single,No
chr2H15971,chr2H15971.1,chr2H:402864281-402873246:+,OUN333_chr2HG06674,OUN333_chr2HG06674.6,ISO&RNA,OUN333,core;multiple,In;Ro;Sh,HOG0014921,core;single,No
chr2H15971,chr2H15971.2,chr2H:402864281-402873246:+,OUN333_chr2HG06674,OUN333_chr2HG06674.2,ISO,OUN333,core;multiple,In;Ro;Sh,HOG0014921,core;single,No
chr2H15971,chr2H15971.3,chr2H:402864281-402873246:+,OUN333_chr2HG06674,OUN333_chr2HG06674.3,ISO,OUN333,core;multiple,In;Ro;Sh,HOG0014921,core;single,No
chr2H15971,chr2H15971.5,chr2H:402864281-402873246:+,OUN333_chr2HG06674,OUN333_chr2HG06674.5,ISO,OUN333,core;multiple,In;Ro;Sh,HOG0014921,core;single,No
chr2H15972,chr2H15972.1,chr2H:402871667-402872292:+,OUN333_chr2HG06674,OUN333_chr2HG06674.7,ISO,OUN333,cloud;single,NotDE,HOG0014921,core;single,No


In [6]:
# subset tpm table with 20let-HOGs
tpm_geneID <- tpm %>% rownames_to_column("PanBaRT20_gene_id")
tpm_filt <- inner_join(hog20genesonly, tpm_geneID, by = "PanBaRT20_gene_id") %>% column_to_rownames("PanBaRT20_gene_id")
dim(tpm_filt)
head(tpm_filt, 2)

[1] 11838   297

,Akashinriki_Ca1,Akashinriki_Ca2,Akashinriki_Ca3,Akashinriki_Co1,Akashinriki_Co2,Akashinriki_Co3,Akashinriki_In1,Akashinriki_In2,Akashinriki_In3,Akashinriki_Ro1,⋯,RGTPlanet_Co3,RGTPlanet_In1,RGTPlanet_In2,RGTPlanet_In3,RGTPlanet_Ro1,RGTPlanet_Ro2,RGTPlanet_Ro3,RGTPlanet_Sh1,RGTPlanet_Sh2,RGTPlanet_Sh3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr6H66186,25.49234,23.29047,29.87313,111.9747,110.7764,119.3126,8.915942,3.067054,18.138828,0,⋯,19.74645,0.082525,0.000000,0.000000,0,0,0,312.31639,274.37789,186.0209
chr7H70574,20.25627,19.78008,23.47470,117.7028,118.0877,124.4923,7.249765,5.715173,6.260837,0,⋯,21.28653,4.672086,2.732484,1.975738,0,0,0,68.48766,51.78224,34.3321


In [7]:
trans_tpm <- as.data.frame(t(tpm))
head(trans_tpm, 2)

,chr1H00001,chr1H00002,chr1H00003,chr1H00004,chr1H00005,chr1H00006,chr1H00007,chr1H00008,chr1H00009,chr1H00010,⋯,chr7H79571,chr7H79572,chr7H79573,chr7H79574,chr7H79575,chr7H79576,chr7H79577,chr7H79578,chr7H79579,chr7H79580
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Akashinriki_Ca1,4.007573,130.42341,0,0,1.808930,5.673198,27.76783,25.03242,8.906746,214.8217,⋯,2.001052,0.713503,1.853821,0.475589,0,10.898248,25.31995,2.112613,0.058038,0
Akashinriki_Ca2,3.927487,95.90984,0,0,1.464428,5.194460,21.77769,24.80091,7.228023,195.0607,⋯,1.397187,0.264392,1.566389,0.209979,0,8.656334,20.34395,1.324194,0.289481,0


In [11]:
# create metatable
meta <- trans_tpm %>% rownames_to_column("ID") %>%
                        select(ID) %>%
                        mutate(Accession = str_extract(ID, regex("[a-z0-9]+", ignore_case = TRUE))) %>%
                        mutate(Tissue = str_sub(str_extract(ID, regex("_[a-z]+", ignore_case = TRUE)), -2)) %>%
                        mutate(Batch = as.numeric(str_extract(ID, regex("[1-3]")))) %>%
                        mutate(Tissue = case_when(Tissue == "Ca" ~ "Caryopsis",
                                                  Tissue == "Co" ~ "Coleoptiles",
                                                  Tissue == "Sh" ~ "Shoot",
                                                  Tissue == "Ro" ~ "Root",
                                                  Tissue == "In" ~ "Inflorescence"))
head(meta, 2)
glimpse(meta)
# save joined metatable for all accessions together
write.csv(meta, file = "PanBaRT20_geneTPM_meta.csv", row.names = FALSE)
# save project table with each accession mentioned once
meta_acc <- meta %>% select(Accession) %>% distinct()
dim(meta_acc)
write.table(meta_acc, file = "PanBaRT20_project_table.txt", row.names = FALSE)
# save accession-wise metatable
acc <- unique(meta$Accession)
for (ac in acc) {
    subset_meta <- meta %>% filter(Accession == ac)
    filename <- paste0(ac, "_meta.csv")
    write.csv(subset_meta, file = filename, row.names = FALSE)
    }

,ID,Accession,Tissue,Batch
,<chr>,<chr>,<chr>,<dbl>
1,Akashinriki_Ca1,Akashinriki,Caryopsis,1
2,Akashinriki_Ca2,Akashinriki,Caryopsis,2


Rows: 297
Columns: 4
$ ID        <chr> "Akashinriki_Ca1", "Akashinriki_Ca2", "Akashinriki_Ca3", "Ak…
$ Accession <chr> "Akashinriki", "Akashinriki", "Akashinriki", "Akashinriki", …
$ Tissue    <chr> "Caryopsis", "Caryopsis", "Caryopsis", "Coleoptiles", "Coleo…
$ Batch     <dbl> 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, …


[1] 20  1

In [23]:
meta$Accession[1]

[1] "Akashinriki"

In [24]:
# divide accessions into separate tpm tables and save them
# create vector of accession types
acc <- unique(meta$Accession)
acc
# Extract subtables for matching column names
subtables <- lapply(acc, function(name) {
  matching_cols <- grep(paste0("^", name), colnames(tpm_filt))
  subtable <- tpm_filt[, matching_cols, drop = FALSE]
  # filter each ort_accession_subtable for low-expressing genes
  idx <- rowSums(subtable >= 0.5) >= 2
  subtable <- subtable[idx,]
  print(paste0(name, " filtered genes with >0.5 for min 2 samples across all samples:", dim(subtable)))
  # Save each subtable in a separate file
  file_name <- paste0("PanBaRT20_geneTPM_ort_filt_", name, ".csv")
  write.csv(subtable, file = file_name, row.names = TRUE)
})

[1] "Akashinriki"   "Barke"         "ZDM02064"      "ZDM01467"     
 [5] "B1K"           "GoldenPromise" "Hockett"       "HOR10350"     
 [9] "HOR13821"      "HOR13942"      "HOR21599"      "HOR3081"      
[13] "HOR3365"       "HOR7552"       "HOR8148"       "HOR9043"      
[17] "Igri"          "Morex"         "OUN333"        "RGTPlanet"

[1] "Akashinriki filtered genes with >0.5 for min 2 samples across all samples:11834"
[2] "Akashinriki filtered genes with >0.5 for min 2 samples across all samples:15"   
[1] "Barke filtered genes with >0.5 for min 2 samples across all samples:11833"
[2] "Barke filtered genes with >0.5 for min 2 samples across all samples:15"   
[1] "ZDM02064 filtered genes with >0.5 for min 2 samples across all samples:11832"
[2] "ZDM02064 filtered genes with >0.5 for min 2 samples across all samples:15"   
[1] "ZDM01467 filtered genes with >0.5 for min 2 samples across all samples:11830"
[2] "ZDM01467 filtered genes with >0.5 for min 2 samples across all samples:15"   
[1] "B1K filtered genes with >0.5 for min 2 samples across all samples:11829"
[2] "B1K filtered genes with >0.5 for min 2 samples across all samples:15"   
[1] "GoldenPromise filtered genes with >0.5 for min 2 samples across all samples:11832"
[2] "GoldenPromise filtered genes with >0.5 for min 2 samples across all samples:15"   
[1] 

In [49]:
sessionInfo()

R version 4.2.3 (2023-03-15)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 22.04.2 LTS

Matrix products: default
BLAS/LAPACK: /home/vanda/miniconda3/envs/r/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_DE.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.2 forcats_1.0.0   stringr_1.5.0   dplyr_1.1.1    
 [5] purrr_1.0.1     readr_2.1.4     tidyr_1.3.0     tibble_3.2.1   
 [9] ggplot2_3.4.1   tidyverse_2.0.0

loaded via a namespace (and not attached):
 [1] pillar_1.9.0     compiler_4.2.3   base64enc_0.1-3  tools_4.2.3     
 [5] digest_0.6.31